In [1]:
%pylab inline

import math
import numpy as np
from scipy.sparse.linalg import inv
#from numpy.linalg import inv
import scipy.sparse as sps
import scipy.sparse.linalg
from scipy import integrate
import sys
import matplotlib.pyplot as plt
sys.path.append('../../src/')
from pylab import *

import parameters as pam
import lattice as lat
import variational_space as vs
import hamiltonian as ham
import basis_change as basis
import get_state as getstate
import utility as util
import plotfig as fig
import ground_state as gs
import lanczos
import time
start_time = time.time()
M_PI = math.pi
                  
#####################################
def compute_Aw_main(ANi,ACu,epCu,epNi,tpd,tpp,tz,pds,pdp,pps,ppp,Upp,\
                    d_Ni_double,d_Cu_double,p_double,double_Ni_part,hole34_Ni_part, double_Cu_part,\
                    hole34_Cu_part, idx_Ni,idx_Cu,U_Ni, S_Ni_val, Sz_Ni_val, AorB_Ni_sym ,U_Cu, S_Cu_val, Sz_Cu_val, AorB_Cu_sym):  
    if Norb==7:
        fname = 'epCu'+str(epCu)+'epNi'+str(epNi)+'_tpd'+str(tpd)+'_tpp'+str(tpp) \
                  +'_Mc'+str(Mc)+'_Norb'+str(Norb)+'_eta'+str(eta) +'_ANi'+str(ANi) \
                  + '_ACu'+str(ACu) + '_B'+str(B) + '_C'+str(C) +'_tz' +str(tz)                  
        flowpeak = 'Norb'+str(Norb)+'_tpp'+str(tpp)+'_Mc'+str(Mc)+'_eta'+str(eta)
    elif Norb==9 or Norb==10 or Norb==11:
        fname = 'epCu'+str(epCu)+'epNi'+str(epNi)+'_pdp'+str(pdp)+'_pps'+str(pps)+'_ppp'+str(ppp) \
                  +'_Mc'+str(Mc)+'_Norb'+str(Norb)+'_eta'+str(eta) \
                  +'_ANi'+str(ANi) + '_ACu'+str(ACu) + '_B'+str(B) + '_C'+str(C)
        flowpeak = 'Norb'+str(Norb)+'_pps'+str(pps)+'_ppp'+str(ppp)+'_Mc'+str(Mc)+'_eta'+str(eta) 
               
                
    w_vals = np.arange(pam.wmin, pam.wmax, pam.eta)
    Aw = np.zeros(len(w_vals))
    Aw_dd_total = np.zeros(len(w_vals))
    Aw_d8_total = np.zeros(len(w_vals))

    # set up H0
    if Norb==7:
        tpd_nn_hop_dir, tpd_orbs, tpd_nn_hop_fac, tpp_nn_hop_fac \
                                   = ham.set_tpd_tpp(Norb,tpd,tpp,0,0,0,0)
    elif Norb==9 or Norb==11:
        tpd_nn_hop_dir, tpd_orbs, tpd_nn_hop_fac, tpp_nn_hop_fac \
                                   = ham.set_tpd_tpp(Norb,0,0,pds,pdp,pps,ppp)
    
    tz_fac = ham.set_tz(Norb,tz)
            
    T_pd   = ham.create_tpd_nn_matrix(VS,tpd_nn_hop_dir, tpd_orbs, tpd_nn_hop_fac)
    T_pp   = ham.create_tpp_nn_matrix(VS,tpp_nn_hop_fac)  
    T_z    = ham.create_tz_matrix(VS,tz_fac)
    Esite  = ham.create_edep_diag_matrix(VS,ANi,ACu,epNi,epCu)      
    
    H0 = T_pd + T_pp + T_z + Esite     
    '''
    Below probably not necessary to do the rotation by multiplying U and U_d
    the basis_change.py is only for label the state as singlet or triplet
    and assign the interaction matrix
    '''
    if pam.if_H0_rotate_byU==1:
        H0_Ni_new = U_Ni_d.dot(H0.dot(U_Ni))
        H0_Cu_new = U_Cu_d.dot(H0.dot(U_Cu))    
    clf()

    if Norb==7 or Norb==9 or Norb==10 or Norb==11:     
        Hint_Ni = ham.create_interaction_matrix_ALL_syms(VS,d_Ni_double,p_double,double_Ni_part, idx_Ni, hole34_Ni_part,  \
                                                      S_Ni_val, Sz_Ni_val,AorB_Ni_sym, ACu, ANi, Upp)
        Hint_Cu = ham.create_interaction_matrix_ALL_syms(VS,d_Cu_double,p_double,double_Cu_part, idx_Cu, hole34_Cu_part, \
                                                      S_Cu_val, Sz_Cu_val,AorB_Cu_sym, ACu, ANi, Upp)        
        
        if pam.if_H0_rotate_byU==1:
           # H = H0_Ni_new +H0_Cu_new + Hint_Ni + Hint_Cu
            H_Ni = H0_Ni_new + Hint_Ni
            H0_Cu_new = U_Cu_d.dot(H_Ni.dot(U_Cu)) 
            H = H0_Cu_new + Hint_Cu
        else:
            H = H0 + Hint_Ni + Hint_Cu
        H.tocsr()

        ####################################################################################
        # compute GS only for turning on full interactions
        if pam.if_get_ground_state==1:
            vals, vecs = gs.get_ground_state(H, VS, S_Ni_val,Sz_Ni_val,S_Cu_val,Sz_Cu_val,tz)
#             if Norb==8:
#                 util.write_GS('Egs_'+flowpeak+'.txt',A,ep,tpd,vals[0])
#             elif Norb==10 or Norb==11 or Norb==12:
#                 util.write_GS2('Egs_'+flowpeak+'.txt',A,ep,pds,pdp,vals[0])
        #########################################################################
        '''
        Compute A(w) for various states
        '''
        if pam.if_compute_Aw==1:
#             # compute d8
#             fig.compute_Aw_d8_sym(H, VS, d_double, S_val, Sz_val, AorB_sym, A, w_vals, "Aw_d8_sym_", fname)

            #compute d9Ld9L
            d9Ld9L_a1L_b1L_state_indices, d9Ld9L_a1L_b1L_state_labels, \
                    = getstate.get_d9Ld9L_state_indices(VS,S_Ni_val, Sz_Ni_val)
            fig.compute_Aw1(H, VS, w_vals,  d9Ld9L_a1L_b1L_state_indices, d9Ld9L_a1L_b1L_state_labels, "Aw_d9Ld9L_a1L_b1L__", fname)
           
        
#             #compute d9d9L2        
#             d9d9L2_a1_b1L2_state_indices, d9d9L2_a1_b1L2_state_labels, \
#                     = getstate.get_d9d9L2_state_indices(VS,S_Ni_val, Sz_Ni_val)
#             fig.compute_Aw1(H, VS, w_vals,  d9d9L2_a1_b1L2_state_indices, d9d9L2_a1_b1L2_state_labels, "Aw_d9d9L2_a1_b1L2__", fname)
#             #compute d9L2d9        
#             d9L2d9_a1L2_b1_state_indices, d9L2d9_a1L2_b1_state_labels, \
#                     = getstate.get_d9L2d9_state_indices(VS,S_Ni_val, Sz_Ni_val)
#             fig.compute_Aw1(H, VS, w_vals,  d9L2d9_a1L2_b1_state_indices, d9L2d9_a1L2_b1_state_labels, "Aw_d9L2d9_a1L2_b1__", fname)
                    
#             fig.compute_Aw_d8d8_sym(H, VS, d_Ni_double, S_Ni_val, Sz_Ni_val, AorB_Ni_sym, ANi, S_Cu_val, \
#                                     Sz_Cu_val,AorB_Cu_sym, ACu, w_vals, "Aw_d8d8_sym_", fname)    
#             fig.compute_Aw_d8Ld9_sym(H, VS, d_double, S_val, Sz_val, AorB_sym, ANi, w_vals, "Aw_d8Ld9_sym_", fname)
#             fig.compute_Aw_d8d9L_sym(H, VS, d_double, S_val, Sz_val, AorB_sym, ANi, w_vals, "Aw_d8d9L_sym_", fname)        
##########################################################################
if __name__ == '__main__': 
    Mc  = pam.Mc
    print ('Mc=',Mc)

    Norb = pam.Norb
    eta  = pam.eta
    edNi = pam.edNi
    edCu = pam.edCu

    ANis = pam.ANis
    ACus = pam.ACus
    B  = pam.B
    C  = pam.C
    
    # set up VS
    VS = vs.VariationalSpace(Mc)
#     basis.count_VS(VS)
    
    d_Ni_double, idx_Ni, hole34_Ni_part,  double_Ni_part, \
    d_Cu_double, idx_Cu, hole34_Cu_part,  double_Cu_part, \
    p_double = ham.get_double_occu_list(VS)
    
    # change the basis for d_double states to be singlet/triplet
    if pam.basis_change_type=='all_states':
        U, S_val, Sz_val, AorB_sym = basis.create_singlet_triplet_basis_change_matrix \
                                    (VS, d_double, double_part, idx, hole34_part)
        if pam.if_print_VS_after_basis_change==1:
            basis.print_VS_after_basis_change(VS,S_val,Sz_val)
    elif pam.basis_change_type=='d_double':
        U_Ni, S_Ni_val, Sz_Ni_val, AorB_Ni_sym = basis.create_singlet_triplet_basis_change_matrix_d_double \
                                    (VS, d_Ni_double, double_Ni_part, idx_Ni, hole34_Ni_part)
        U_Cu, S_Cu_val, Sz_Cu_val, AorB_Cu_sym = basis.create_singlet_triplet_basis_change_matrix_d_double \
                                    (VS, d_Cu_double, double_Cu_part, idx_Cu, hole34_Cu_part)
#         U = U_Ni+U_Cu
#         print(U_Ni)
#         print(U_Cu)    
    U_Ni_d = (U_Ni.conjugate()).transpose()
    U_Cu_d = (U_Cu.conjugate()).transpose()    
    # check if U if unitary
#     util.checkU_unitary(U_Ni,U_Ni_d)
    
    if Norb==7:
        for a in range(0,len(pam.tzs)):
            tz = pam.tzs[a] 
            for tpd in pam.tpds:
                for epCu in pam.epCus:
                    for epNi in pam.epNis:               
                        for ANi in pam.ANis:
                            for ACu in pam.ACus:
    #                            util.get_atomic_d8_energy(ANi,B,C)
                                for tpp in pam.tpps:
                                    for Upp in pam.Upps:
                                        print ('===================================================')
                                        print ('ANi=',ANi, 'ACu=',ACu,'epCu=', epCu, 'epNi=',epNi,\
                                               ' tpd=',tpd,' tpp=',tpp,' Upp=',Upp ,'tz=',tz)

                                        compute_Aw_main(ANi,ACu,epCu,epNi,tpd,tpp,tz,0,0,0,0,Upp,\
                                                        d_Ni_double,d_Cu_double,p_double,double_Ni_part,hole34_Ni_part,\
                                                        double_Cu_part,hole34_Cu_part, idx_Ni,idx_Cu, \
                                                        U_Ni, S_Ni_val, Sz_Ni_val, AorB_Ni_sym ,U_Cu, S_Cu_val, Sz_Cu_val, AorB_Cu_sym)  
    elif Norb==9 or Norb==10 or Norb==11:
        pps = pam.pps
        ppp = pam.ppp
        for a in range(0,len(pam.tzs)):
            tz = pam.tzs[a]                                        
            for ii in range(0,len(pam.pdps)):
                pds = pam.pdss[ii]
                pdp = pam.pdps[ii]
                for epCu in pam.epCus:
                    for epNi in pam.epNis:
                        for ANi in pam.ANis: 
                            for ACu in pam.ACus:
    #                         util.get_atomic_d8_energy(ANi,B,C)
                                for Upp in pam.Upps:
                                    print ('===================================================')
                                    print ('ANi=',ANi, 'ACu=',ACu,'epCu=',epCu, 'epNi=',epNi,' pds=',pds,\
                                           ' pdp=',pdp,' pps=',pps,' ppp=',ppp,' Upp=',Upp,'tz=',tz)
                                    compute_Aw_main(ANi,ACu,epCu,epNi,0,0,tz,pds,pdp,pps,ppp,Upp,\
                                                   d_Ni_double,d_Cu_double,p_double,double_Ni_part,hole34_Ni_part,\
                                                   double_Cu_part,hole34_Cu_part,idx_Ni,idx_Cu,\
                                                   U_Ni, S_Ni_val, Sz_Ni_val, AorB_Ni_sym ,U_Cu, S_Cu_val, Sz_Cu_val, AorB_Cu_sym) 

                        
    print("--- %s seconds ---" % (time.time() - start_time))

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib
Ni_Cu_orbs =  ['d3z2r2', 'dx2y2', 'dxy', 'dxz', 'dyz']
O1_orbs =  ['px']
O2_orbs =  ['py']
compute A(w) for symmetries =  ['1A1', '3B1']
Mc= 2
VS.dim =  20609
len(d_Ni_list) 5785
len(d_Cu_list) 5785
len(p_list) 2312
len(idx_Ni) 5785
len(idx_Cu) 5785
ANi= 6.0 ACu= 6.0 epCu= 7.0 epNi= 7.0  tpd= 1.3  tpp= 0.55  Upp= 0 tz= 0.01
start create_tpd_nn_matrix
start create_tpp_nn_matrix
start create_tz_matrix
start create_edep_diag_matrix
start create_interaction_matrix
--- create_interaction_matrix_ALL_syms 2.699772834777832 seconds ---
start create_interaction_matrix
--- create_interaction_matrix_ALL_syms 2.780529499053955 seconds ---
start getting ground state
lowest eigenvalue of H from np.linalg.eigsh = 
[4.27837816 4.39503293 4.39518686 4.51188976 4.51192548 4.51199691
 4.95907269 4.95907269 4.95907269 4.95907269]
eigenvalue =  4.278378156702169
Com

 state  3088 dxz up 0 0 1 dxz dn 0 0 1 dx2y2 dn 0 0 0 px up -1 0 0 SNi= 0 SzNi= 0 SCu= 0 SzCu= 0 , weight =  0.0006053721043749355
 state  3089 dxz up 0 0 1 dxz dn 0 0 1 dx2y2 up 0 0 0 px dn -1 0 0 SNi= 0 SzNi= 0 SCu= 0 SzCu= 0 , weight =  0.000605372104374929
 state  3642 dx2y2 dn 0 0 1 dxz up 0 0 0 dxz dn 0 0 0 px up -1 0 1 SNi= 0 SzNi= 0 SCu= 0 SzCu= 0 , weight =  0.0006053721043749297
 state  3643 dx2y2 up 0 0 1 dxz up 0 0 0 dxz dn 0 0 0 px dn -1 0 1 SNi= 0 SzNi= 0 SCu= 0 SzCu= 0 , weight =  0.0006053721043749368
 state  4066 dx2y2 dn 0 0 1 d3z2r2 up 0 0 0 d3z2r2 dn 0 0 0 py up 0 -1 1 SNi= 0 SzNi= 0 SCu= 0 SzCu= 0 , weight =  0.0006762301961487589
 state  4067 dx2y2 up 0 0 1 d3z2r2 up 0 0 0 d3z2r2 dn 0 0 0 py dn 0 -1 1 SNi= 0 SzNi= 0 SCu= 0 SzCu= 0 , weight =  0.0006762301961487557
 state  4304 d3z2r2 up 0 0 1 d3z2r2 dn 0 0 1 dx2y2 dn 0 0 0 py up 0 -1 0 SNi= 0 SzNi= 0 SCu= 0 SzCu= 0 , weight =  0.0006762301961487563
 state  4305 d3z2r2 up 0 0 1 d3z2r2 dn 0 0 1 dx2y2 up 0 0 0 py dn 

 state  11299 dx2y2 dn 0 0 1 py up 0 -1 1 py up 0 1 0 py dn 0 1 0 SNi= 0 SzNi= 0 SCu= 0 SzCu= 0 , weight =  0.00043485271043881553
 state  11300 dx2y2 up 0 0 1 py dn 0 -1 1 py up 0 1 0 py dn 0 1 0 SNi= 0 SzNi= 0 SCu= 0 SzCu= 0 , weight =  0.0004348527104388119
 state  11307 dx2y2 up 0 0 0 py up 0 -1 1 py dn 0 1 1 py dn 0 1 0 SNi= 0 SzNi= 0 SCu= 0 SzCu= 0 , weight =  0.00043485271043881363
 state  11309 dx2y2 up 0 0 0 py dn 0 -1 1 py up 0 1 1 py dn 0 1 0 SNi= 0 SzNi= 0 SCu= 0 SzCu= 0 , weight =  0.00043485271043881303
 state  11310 dx2y2 dn 0 0 0 py up 0 -1 1 py dn 0 1 1 py up 0 1 0 SNi= 0 SzNi= 0 SCu= 0 SzCu= 0 , weight =  0.0004348527104388113
 state  11312 dx2y2 dn 0 0 0 py dn 0 -1 1 py up 0 1 1 py up 0 1 0 SNi= 0 SzNi= 0 SCu= 0 SzCu= 0 , weight =  0.00043485271043881103
 state  11313 dx2y2 up 0 0 1 py dn 0 1 1 py up 0 -1 0 py dn 0 1 0 SNi= 0 SzNi= 0 SCu= 0 SzCu= 0 , weight =  0.00043485271043881217
 state  11314 dx2y2 dn 0 0 1 py up 0 1 1 py up 0 -1 0 py dn 0 1 0 SNi= 0 SzNi= 0 SCu=

 state  14403 dx2y2 dn 0 0 1 dx2y2 up 0 0 0 py up 0 -1 1 px dn 1 0 0 SNi= 0 SzNi= 0 SCu= 0 SzCu= 0 , weight =  0.006218335000627362
 state  14404 dx2y2 up 0 0 1 dx2y2 up 0 0 0 py dn 0 -1 1 px dn 1 0 0 SNi= 0 SzNi= 0 SCu= 0 SzCu= 0 , weight =  0.006218377807573524
 state  14405 dx2y2 dn 0 0 1 dx2y2 dn 0 0 0 py up 0 -1 1 px up 1 0 0 SNi= 0 SzNi= 0 SCu= 0 SzCu= 0 , weight =  0.00621837780757359
 state  14406 dx2y2 up 0 0 1 dx2y2 dn 0 0 0 py dn 0 -1 1 px up 1 0 0 SNi= 0 SzNi= 0 SCu= 0 SzCu= 0 , weight =  0.006218335000627347
 state  14408 dx2y2 up 0 0 1 dx2y2 dn 0 0 1 py up 0 -1 0 px dn 1 0 0 SNi= 0 SzNi= 0 SCu= 0 SzCu= 0 , weight =  0.0015382014285215614
 state  14409 dx2y2 up 0 0 1 dx2y2 dn 0 0 1 py dn 0 -1 0 px up 1 0 0 SNi= 0 SzNi= 0 SCu= 0 SzCu= 0 , weight =  0.0015382014285215682
 state  14414 dx2y2 up 0 0 0 dx2y2 dn 0 0 0 py up 0 -1 1 px dn 1 0 1 SNi= 0 SzNi= 0 SCu= 0 SzCu= 0 , weight =  0.001538201428521574
 state  14415 dx2y2 up 0 0 0 dx2y2 dn 0 0 0 py dn 0 -1 1 px up 1 0 1 SNi= 0

 state  17513 dx2y2 dn 0 0 1 px up 1 0 1 py dn 0 -1 0 py up 0 1 0 SNi= 0 SzNi= 0 SCu= 0 SzCu= 0 , weight =  0.0004348509145515934
 state  17521 dx2y2 dn 0 0 0 py up 0 1 1 px dn 1 0 1 py up 0 -1 0 SNi= 0 SzNi= 0 SCu= 0 SzCu= 0 , weight =  0.00046620792393551274
 state  17522 dx2y2 up 0 0 0 py up 0 1 1 px dn 1 0 1 py dn 0 -1 0 SNi= 0 SzNi= 0 SCu= 0 SzCu= 0 , weight =  0.0004662193474291452
 state  17523 dx2y2 dn 0 0 0 py dn 0 1 1 px up 1 0 1 py up 0 -1 0 SNi= 0 SzNi= 0 SCu= 0 SzCu= 0 , weight =  0.00046621934742913996
 state  17524 dx2y2 up 0 0 0 py dn 0 1 1 px up 1 0 1 py dn 0 -1 0 SNi= 0 SzNi= 0 SCu= 0 SzCu= 0 , weight =  0.00046620792393550645
 state  18096 dx2y2 up 0 0 1 dx2y2 dn 0 0 1 py up 0 1 0 px dn 1 0 0 SNi= 0 SzNi= 0 SCu= 0 SzCu= 0 , weight =  0.0015382014285215625
 state  18097 dx2y2 up 0 0 1 dx2y2 dn 0 0 1 py dn 0 1 0 px up 1 0 0 SNi= 0 SzNi= 0 SCu= 0 SzCu= 0 , weight =  0.0015382014285215708
 state  18100 dx2y2 up 0 0 1 dx2y2 up 0 0 0 py dn 0 1 1 px dn 1 0 0 SNi= 0 SzNi= 0 

sumweight2= 0.9999999953676851
sumweight_picture= 0.9887402622333797
LmLn= 0.008356365901322176
d9Ld10L2= 0.05777924418350547
d9d10L3= 4.632253910512996e-09
d9L2d10L= 6.8058556441139095e-09
d10Ld9L2= 6.8058556441173506e-09
d10d9L3= 6.637423638877227e-14
d10L2d9L= 0.057779244183505524
d8Ld10L= 2.2777357114297565e-08
d10Ld8L= 2.2777357114300046e-08
d8d10L2= 0.024567352909901058
d10d8L2= 3.622145100842269e-13
d8L2d10= 3.6221451008237014e-13
d10L2d8= 0.024567352909901044
d9L2d9= 2.1849982199819793e-07
d9d9L2= 2.1849982199826223e-07
d9Ld9L= 0.3995093965315203
d9d8L= 1.6646396230392878e-07
d8d9L= 0.16985884206137397
d9Ld8= 0.1698588420613741
d8Ld9= 1.664639623038425e-07
d8d8= 0.0764627877639373
LmLn= [0.00835637 0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]
d9Ld10L2= [0.05766819 0.00011106 0.05777924 0.         0.         0.
 0.         0.         0.05777924 0.        ]
d9d10L3= [2.64955176e-10 4.36729873e-09 4.63225391e-09 0.00000000e+00
 0.000

<Figure size 432x288 with 0 Axes>